# Using `PairID` to extract PSA data

Here we will use the convenience class `PSAIdentifier` to extract Hausdorff pair analysis data generated by PSA.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Suppress FutureWarning about element-wise comparison to None
# Occurs when calling PSA plotting functions
import warnings
warnings.filterwarnings('ignore')

## Set up input data for `PSA` using `MDAnalysis`

In [2]:
from MDAnalysis import Universe
from MDAnalysis.analysis.psa import PSAnalysis
from pair_id import PairID

Initialize lists for the methods on which to perform PSA. PSA will be performed for four different simulations methods with three runs for each: **DIMS**, **FRODA**, **rTMD-F**, and **rTMD-S**. Also initialize a `PSAIdentifier` object to keep track of the data corresponding to comparisons between pairs of simulations.

In [3]:
method_names = ['DIMS','FRODA','rTMD-F','rTMD-S']
labels = [] # Heat map labels (not plotted in this example)
simulations = [] # List of simulation topology/trajectory filename pairs
universes = [] # List of MDAnalysis Universes representing simulations

For each method, get the topology and each of three total trajectories (per method). Each simulation is represented as a `(topology, trajectory)` pair of file names, which is appended to a master list of simulations.

In [4]:
for method in method_names:
    # Note: DIMS uses the PSF topology format
    topname = 'top.psf' if 'DIMS' in method or 'TMD' in method else 'top.pdb'
    pathname = 'fitted_psa.dcd'
    method_dir = 'methods/{}'.format(method)
    if method is not 'LinInt':
        for run in xrange(1, 4): # 3 runs per method
            run_dir = '{}/{:03n}'.format(method_dir, run)
            topology = '{}/{}'.format(method_dir, topname)
            trajectory = '{}/{}'.format(run_dir, pathname)
            labels.append(method + '(' + str(run) + ')')
            simulations.append((topology, trajectory))
    else: # only one LinInt trajectory
        topology = '{}/{}'.format(method_dir, topname)
        trajectory = '{}/{}'.format(method_dir, pathname)
        labels.append(method)
        simulations.append((topology, trajectory))

Generate a list of universes from the list of simulations.

In [5]:
for sim in simulations:
    universes.append(Universe(*sim))

## Perform a path similarity analysis

Initialize a PSA comparison from the universe list using a C$_\alpha$ trajectory representation.

In [6]:
psa_hpa = PSAnalysis(universes, path_select='name CA', labels=labels)

Generate PSA `Path`s from the trajectories

In [7]:
psa_hpa.generate_paths()

Perform a Hausdorff pairs analysis on all of the `Path`s

In [8]:
psa_hpa.run_pairs_analysis(hausdorff_pairs=True, neighbors=True)

## Extract specific data from PSA

In [9]:
identifier = PairID()
for name in method_names:
    identifier.add_sim(name, [1,2,3])

Get the PSA ID for the second DIMS simulation (DIMS 2) and third rTMD-F simulation (rTMD-F 3)

In [10]:
pid = identifier.get_pair_id('DIMS 2', 'rTMD-F 3')

### Use the PSA ID to locate the Hausdorff analysis data for the DIMS 2/rTMD-F 3 comparison:

Get the indices of the frames for the DIMS 2 and rTMD-F 3 paths corresponding to the Hausdorff pair

In [11]:
psa_hpa.all_hausdorff_pairs[pid]

{'distance': 1.8656396, 'frames': (48, 97)}

In [12]:
psa_hpa.all_hausdorff_pairs[pid]['frames']

(48, 97)

Get the rmsd separating the Hausdorff pair (this is the Hausdorff distance!)

In [13]:
psa_hpa.all_hausdorff_pairs[pid]['distance']

1.8656396

Get the indices of the nearest neighbor frames for the DIMS 2 and rTMD-F 3 paths

In [14]:
psa_hpa.all_nearest_neighbors[pid]['frames']

(array([  3,   3,   7,   8,  13,  13,  17,  17,  19,  22,  22,  22,  28,
         29,  30,  36,  36,  36,  37,  42,  42,  43,  49,  51,  52,  56,
         57,  57,  57,  57,  63,  63,  68,  68,  68,  69,  76,  80,  80,
         83,  84,  84,  89,  90,  90,  91,  91,  93,  97, 102, 102, 105,
        105, 109, 110, 110, 115, 116, 116, 116, 116, 116, 117, 125, 127,
        127, 127, 129, 129, 129, 129, 130, 129, 133, 133, 133, 133, 133,
        133, 134, 133, 133, 137, 138, 134, 138, 138, 138, 138, 139, 138, 139]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  3,
         3,  3,  3,  3,  3,  3,  4,  4,  4, 10, 10, 10, 10, 10, 10, 12, 12,
        14, 14, 14, 14, 15, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 22, 22, 26, 29, 29, 29, 29, 29, 29, 29, 29, 34, 34,
        34, 34, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 36, 37, 37, 42, 42,
        42, 42, 42, 42, 44, 44, 44, 45, 45, 45, 45, 45, 45, 45, 45, 52, 52,
        54, 58, 59, 61, 61

Get the nearest neighbor rmsds for the paths

In [15]:
psa_hpa.all_nearest_neighbors[pid]['distances']

(array([ 0.61047804,  0.72486514,  0.78072226,  0.83813328,  0.93167013,
         1.01148224,  1.06094921,  1.11567676,  1.15830064,  1.18572509,
         1.19752169,  1.24201703,  1.28473079,  1.3059119 ,  1.32284486,
         1.34145117,  1.36143172,  1.36866331,  1.45567334,  1.54358423,
         1.5964545 ,  1.63075888,  1.64360261,  1.68257856,  1.72496045,
         1.71841514,  1.71702886,  1.72773194,  1.76599729,  1.72124612,
         1.77787733,  1.76189709,  1.75216496,  1.74139595,  1.7261765 ,
         1.73064542,  1.76115847,  1.7780937 ,  1.8280741 ,  1.81312251,
         1.81970263,  1.79710293,  1.77227151,  1.7905525 ,  1.76451468,
         1.77083576,  1.80586505,  1.83057499,  1.86563957,  1.85736096,
         1.86288309,  1.82903266,  1.7895925 ,  1.78329408,  1.74680781,
         1.72502816,  1.72101748,  1.66805542,  1.63964784,  1.60171819,
         1.57724261,  1.5326587 ,  1.55232775,  1.51061881,  1.47200549,
         1.44320154,  1.38229811,  1.31344652,  1.2

Display the `pandas` `DataFrame` containing the set of simulations

In [16]:
df = identifier.data
df

Sim ID
Name   Run ID        
DIMS   1            0
       2            1
       3            2
FRODA  1            3
       2            4
       3            5
rTMD-F 1            6
       2            7
       3            8
rTMD-S 1            9
       2           10
       3           11

Get the simulation IDs for DIMS simulations 1, 2, and 3

In [17]:
df.loc[('DIMS',[1,2,3]), 'Sim ID']

Name  Run ID
DIMS  1         0
      2         1
      3         2
Name: Sim ID, dtype: int64